# Λήψη περιβαλλοντικών δεδομένων από [openWeather](https://openweathermap.org/) και αποστολή σε [MQTT Broker](https://www.hivemq.com/public-mqtt-broker/)

Δεδομένης της φύσης του συγκριμένου workshop και στη προσπάθεια να μελετήσουμε πως μπορούμε να διαβάσουμε δεδομένα από έναν αισθητήρα και να τα στείλουμε σε ένα εξυπηρετητή επικοινωνίας μηχανών θα υποθέσουμε το εξής σενάριο: 
1. **Αντί για κάποιον αισθητήρα** έστω ότι τα περιβαλλοντικά δεδομένα τα λαμβάνουμε από κάποια υπηρεσία όπως το openWeather.
<img src="https://www.codeunderscored.com/wp-content/uploads/2021/07/Getting-weather-report-using-OpenWeather-Map-API.png" width="250">

2. Τα δεδομένα που λαμβάνουμε από την υπηρεσία **τα στέλνουμε σε κάποιον mqtt Broker** και έστω ότι αντί για το AWS IoT Core τα στέλνουμε σε έναν public mqtt broker όπως ο HiveMQ public broker
<img src="https://www.hivemq.com/img/connect-any-device.gif" width="250">


In [1]:
# Εγκατάσταση στην εικονική μηχανή του Google Colab του openWeather API
!pip install pyowm # Βιβλιοθήκη η οποία περιέχει το openWeather data API

     |████████████████████████████████| 3.1 MB 7.1 MB/s 


> Εισαγωγή της βιβλιοθήκης `pyowm` προκειμένου να κάνουμε χρήση των μεθόδων της για την λήψη των περιβαλλοντικών δεδομένων.

In [2]:
import pyowm # import Python Open Weather Map to our project.

## Δημιουργία λογαριασμού στο openWeather

Προκειμένου να μπορέσουμε να λάβουμε δεδομένα από την υπηρεσία openWeather θα πρέπει να δημιουργήσουμε έναν `free` λογαριασμό. Η διαδικασία για να το κάνουμε αυτό περιγράφετε στα παρακάτω βήματα:
1. Επισκεπτόμαστε τη παρακάτω διεύθυνση στην οποία θα μας δωθεί η δυνατότητα να δημιουργήσουμε το νέο λογαριασμό (βλέπε παρακάτω σχήμα),
https://openweathermap.org/
![](https://pythonhowtoprogram.com/wp-content/uploads/2020/08/image-12.png)

2. Για να ενεργοποιηθεί το openweather API θα πρέπει να κάνουμε επιβεβαίωση της δημιουργία του λογαριασμού μέσα από το email που θα λάβουμε,
![](https://pythonhowtoprogram.com/wp-content/uploads/2020/08/image-13.png)

3. Έπειτα από την εκτέλεση των παραπάνω βημάτων είμαστε σε θέση να έχουμε πρόσβαση στο `API key` μας μέσα από το σχετικό πεδίο του GUI in your dash board.
![](https://pythonhowtoprogram.com/wp-content/uploads/2020/08/image-14.png)



In [3]:
#@title Εισαγωγή του `API Key` και δημιουργία ενός `weather_manager` αντικειμένου { vertical-output: true }
APIKEY='4919d9b07d4e965e83709916c71a4c79'    #your API Key here as string
OpenWMap=pyowm.OWM(APIKEY)                   # Use API key to get data

mgr = OpenWMap.weather_manager()
Weather = mgr.weather_at_place('Patras')

#Weather=OpenWMap.weather_at_place('London')  # give where you need to see the weather
Data = Weather.weather                        # get out data in the mentioned location

In [4]:
#@title Παράδειγμα εξαγωγής περιβαλλοντικών πληροφοριών από το `openWeather` { vertical-output: true }
temp = Data.temperature(unit='celsius')      # get current temparature in celsius 
print ("Average Temp. Currently ", temp['temp']) # get avg. tmp
print ("Max Temp. Currently ", temp['temp_max']) # get max tmp
print ("Min Temp. Currently ", temp['temp_min']) # get min tmp>>
# ---------------------------------------------------------------
humidity = Data.humidity # get current humidity 
print ("Humidity : ",humidity) # print humidity 
# ---------------------------------------------------------------
wind = Data.wind() # get current wind 
print ("Wind Speed : ",wind['speed']) # print wind speed
print ("Wind Direction in Deg : ",wind['deg']) # print wind Direction
# ----------------------------------------------------------------
cloud = Data.clouds # get current cloud 
print ("Cloud Coverage Percentage : ",cloud) # print cloud coverage percentage

Average Temp. Currently  18.27
Max Temp. Currently  18.99
Min Temp. Currently  15.97
Humidity :  73
Wind Speed :  4.6
Wind Direction in Deg :  64
Cloud Coverage Percentage :  100


In [5]:
#@title Άλλες παράμετροι { vertical-output: true }
print(Data.sunset_time())   # get sunset time
print(Data.sunrise_time())  # get sunrise time
print(Data.pressure)        # get pressure data 
print(Data.status)          # get status of current weather    eg: Rain
print(Data.detailed_status) # get status with more detailed  eg: light rain

1636471564
1636434468
{'press': 1022, 'sea_level': 1022}
Clouds
overcast clouds


start='2016-07-01 15:00:00Z' and interval='hour': searches from 3 to 4 PM of day 2016-07-01

start='2016-07-01' and interval='day': searches on the day 2016-07-01

start='2016-07-01' and interval='month': searches on the month of July 2016

start='2016-07-01' and interval='year': searches from day 2016-07-01 up to the end of year 2016


https://api.openweathermap.org/data/2.5/air_pollution/history?lat=30&lon=50&start=1606223802&end=1606482999&appid=9ad805b30a470f3bd940de9227fb29c4

```python
from pyowm.utils import timestamps


# get an air pollution manager object
mgr = OpenWMap.airpollution_manager()

lon = 20.8
lat = 30.9
# Get latest CO Index on geocoordinates
#coi = mgr.coindex_around_coords(lat, lon)

# Get available CO Index in the last 24 hours
coi = mgr.coindex_around_coords(lat, lon,
    start=timestamps.yesterday(), interval='day')
```



https://www.hivemq.com/public-mqtt-broker/


## Δημοσίευση `(publishing)`  και εγγραφή `(subscribing)` μέσα από το εξυπηρετητή **MQTT σε python**

https://www.hivemq.com/public-mqtt-broker/

In [6]:
#@title Εγκατάσταση της βιβλιοθήκης `paho-mqtt` για την χρήση MQTT στη Python { vertical-output: true }
!pip install paho-mqtt

     |████████████████████████████████| 99 kB 4.7 MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62133 sha256=7ece34aca80865dd2ea610afb78edb29140e8b5262dfee1ca48e4ca5e499232e
  Stored in directory: /root/.cache/pip/wheels/d0/bf/ac/2b3f43f8c6fcd0f4ba5395397458c521eb0b52d33b574a5a40
Successfully built paho-mqtt


In [7]:
#@title Εισαγωγή της βιβλιοθήκης `paho.mqtt.client`
import paho.mqtt.client as mqtt

**Creating a Client Instance**
> The Client class has a few optional arguments that you can specify

> `Client(client_id=””, clean_session=True, userdata=None, protocol=MQTTv311, transport=”tcp”)`
>
You can replace the client_name argument with any name, if you leave it empty it will generate a random id


In [9]:
#@title Δημιουργία ενός αντικειμένου πελάτη για τη σύνδεση με τον mqtt Broker
#@markdown Η μέθοδος για τη δημιουργία του εξυπηρετητή έχει τη παρακάτω σύνταξη
#@markdown `Client(client_id=””, clean_session=True, userdata=None, protocol=MQTTv311, transport=”tcp”)`
#@markdown Μπορούμε να δώσουμε ότι θέλουμε ως `client_name`, αν το αφήσουμε κενό θα δώσει αυτόματα μια τυχαία ταυτότητα `random id`
client_name=""
client =mqtt.Client(client_name)

In [10]:
#@title **Σύνδεση με τον `mqtt broker` ή `server`** { vertical-output: true }
#@markdown **Δημοσίευση μηνυμάτων (publishing Messages):**
#@markdown Με το που γίνει η σύνδεση μπορεί να αρχίσει η δημοσίευση μηνυμάτων.
#@markdown Η μέθοδος `publish` δέχεται `4 παραμέτρους` οι οποίοι φαίνονται παρακάτω με τις προεπιλεγμένες τιμές:
#@markdown
#@markdown > `publish(topic, payload=None, qos=0, retain=False)`

#@markdown **Ορίσματα σύνδεσης**
host = "broker.hivemq.com" #@param {type:"string"} # Replace this with any host_name or IP adress
topic = "weather_data" #@param {type:"string"} # Replace this with any host_name or IP adress

# -- Σύνδεση με mqtt broker
client.connect(host);
message = temp['temp']
# -- Αποστολή μηνύματος στο topic του server
client.publish(topic,message)
print("Published message: " + str(temp['temp']))

Published message: 18.27


**Subscribing To Topics**

>To subscribe to a topic you use the subscribe method of the Paho MQTT Class object.

>The subscribe method accepts 2 parameters – A topic or topics and a QOS (quality of Service) as shown below with their default values.

>`subscribe(topic, qos=0)`

In [ ]:
client.subscribe(topic)

(0, 2)

When the client receives messages it triggers the `on_message` callback function.

To view those messages we need to activate and process the on_message callback.

```
def on_message(client, userdata, message):
    print("message received " ,str(message.payload.decode("utf-8")))
    print("message topic=",message.topic)
    print("message qos=",message.qos)
    print("message retain flag=",message.retain)
```



Then we need to attach the `on_message` callback to our client object

> `client.on_message=on_message        #attach function to callback`

Finally we need to run a loop otherwise we won’t see the callbacks

> `client.loop_start()    #start the loop`

**Publish and Subscribe example script**

In [12]:
#@title Ολοκληρωμένο παράδειγμα σύνδεσης και δημοσίευσης σε mqtt broker { vertical-output: true }
broker_address="broker.hivemq.com" #@param {type:"string"}
topic = "weather_data" #@param {type:"string"}
location = "Patras"  #@param {type:"string"}
unit_fmt = "celsius"  #@param {type:"string"}

# -- Εισαγωγή βιβλιοθηκών -------------------------------------------
import paho.mqtt.client as mqtt
import pyowm # import Python Open Weather Map to our project.
import json #import the JSON library
import time

# -- Συνάρτηση η οποία ΧΧΧΧΧ -------------------------------------------
def on_message(client, userdata, message):
    print("message received " ,str(message.payload.decode("utf-8")))
    print("message topic=",message.topic)

# -- openWeather API Key ----------------------------------------------
APIKEY='4919d9b07d4e965e83709916c71a4c79' #your API Key here as string

# -- Δημιουργία αντικειμένου openweather ------------------------------
OpenWMap=pyowm.OWM(APIKEY) # Use API key to get data
# ---- Λήψη δεδομένων για συγκεκριμένη τοποθεσία
mgr = OpenWMap.weather_manager()
Weather = mgr.weather_at_place(location) #Replace with any city name

Data = Weather.weather # get out data in the mentioned location

temp = Data.temperature(unit=unit_fmt)
pressure = Data.pressure
# Organise data in a python dictionary
weather_data = {
  "Average Temperature ":temp['temp'],
  "Atmospheric Pressure": pressure['press'],
  "Sea Level Pressure": pressure['sea_level'],
  "Current Weather Status": Data.detailed_status
}

messageJSON = json.dumps(weather_data) #covert to JSON format

print("creating new instance")
client = mqtt.Client("") #create new instance
client.on_message=on_message #attach function to callback
print("connecting to broker")
client.connect(broker_address) #connect to broker
client.loop_start() #start the loop
print("Subscribing to topic",topic)
client.subscribe(topic)
print("Publishing JSON message to topic",topic)
client.publish(topic, messageJSON)
time.sleep(4) # wait
client.loop_stop() #stop the loop

creating new instance
connecting to broker
Subscribing to topic weather_data
Publishing JSON message to topic weather_data
message received  {"Average Temperature ": 18.27, "Atmospheric Pressure": 1022, "Sea Level Pressure": 1022, "Current Weather Status": "overcast clouds"}
message topic= weather_data
